In [ ]:
import perceval as pcvl
from perceval.algorithm import Sampler
from math import pi
from perceval.components import BS, PS, PERM
import numpy as np
from perceval import Encoding, Port

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Generate dataset with 2 features
X, y = make_classification(n_samples=100, n_features=2, n_classes=2, n_clusters_per_class=1, n_redundant = 0,random_state=42)

# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Plot the dataset
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, marker='o', label="Train", alpha=0.6)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, marker='s', label="Test", alpha=0.6)
plt.legend()
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.title("Generated Classification Dataset")
plt.show()


In [ ]:
#KLM CNOT

def fock_states(n, m):
    """Return a list of lists corresponding to all Fock states with
    n photons in m modes, ordered in descending lexicographic order."""
    if m == 1:
        return [[n]]
    states = []
    # Iterate i from n down to 0 to ensure descending order in the first mode
    for i in range(n, -1, -1):
        for tail in fock_states(n - i, m - 1):
            states.append([i] + tail)
    return states


state_00 = pcvl.BasicState([1, 0, 1, 0])
state_01 = pcvl.BasicState([1, 0, 0, 1])
state_10 = pcvl.BasicState([0, 1, 1, 0])
state_11 = pcvl.BasicState([0, 1, 0, 1])
state_matrix = np.array([np.array(state_00), np.array(state_01),np.array(state_10), np.array(state_11)])

states = np.array(fock_states(10, 20))
result = states[:,:4] @ state_matrix
max_indices = result.argmax(1)
indices_to_use = np.where(max_indices % 2 == 1)[0]
indices_to_use.shape
keys = [pcvl.BasicState(states[i]) for i in indices_to_use]

In [ ]:
U = np.array([[ 1-np.sqrt(2),              2**-0.25,             (3/np.sqrt(2) - 2)**0.5],
              [ 2**-0.25,                   0.5,                        0.5-np.sqrt(0.5)],
              [ (3/np.sqrt(2) - 2)**0.5,     0.5-np.sqrt(0.5),          np.sqrt(2)-0.5]])
NS = pcvl.Circuit.decomposition(U, BS.Ry(theta=pcvl.P('theta'), phi_tr=0), phase_shifter_fn=PS, inverse_v=True)

KLM_CZ = (pcvl.Circuit(8)
              .add(0, PERM([0, 4, 2, 3, 1, 5, 6, 7]))
              .add(0, BS.Ry(np.pi/2), merge=False)
              .add(0, PERM([0, 4, 2, 3, 1, 5, 6, 7]))
              .add(0, NS, merge=False)
              .add(4, NS, merge=False)
              .add(0, PERM([0, 4, 2, 3, 1, 5, 6, 7]))
              .add(0, BS.Ry(-np.pi/2), merge=False)
              .add(0, PERM([0, 4, 2, 3, 1, 5, 6, 7])))
KLM_CZ_P = pcvl.Processor("SLOS", KLM_CZ)
KLM_CZ_P.add_herald(1, 1)\
            .add_herald(2, 0)\
            .add_herald(5, 1)\
            .add_herald(6, 0)

Theta_45 = np.pi/2
CZ_TP1 = (pcvl.Processor("SLOS", 8)
            .add_port(0, Port(Encoding.DUAL_RAIL, 'Q1')) \
            .add_port(2, Port(Encoding.DUAL_RAIL, 'Q2')) \
            .add(0, PERM([0, 1, 6, 7, 2, 3, 4, 5])) \
            .add((2, 3), BS.Ry(-Theta_45)) \
            .add((4, 5), BS.Ry(Theta_45)) \
            .add(0, PERM([0, 1, 3, 2, 4, 5, 7, 6])) \
            .add(2, KLM_CZ_P) \
            .add(0, PERM([0, 1, 3, 2, 4, 5, 6, 7])) \
            .add((1, 2), BS.Ry(Theta_45)) \
            .add((5, 6), BS.Ry(-Theta_45))) \
            .add((1,2,3), PERM([2,1,0])) \
            .add((4,5,6), PERM([2,1,0]))

ff_z1 = pcvl.FFCircuitProvider(2,-4,pcvl.Circuit(1))
ff_z1.add_configuration([1,0], PS(np.pi))

ff_z2 = pcvl.FFCircuitProvider(2,-3,pcvl.Circuit(1))
ff_z2.add_configuration([0,1], PS(np.pi))

CZ_TP1.add(0, PERM([0, 1, 4, 5, 6, 7, 2, 3]))

CZ_TP1.add(4,pcvl.Detector.pnr())
CZ_TP1.add(5,pcvl.Detector.pnr())
CZ_TP1.add(6,pcvl.Detector.pnr())
CZ_TP1.add(7,pcvl.Detector.pnr())

CZ_TP1.add(4,ff_z2)
CZ_TP1.add(6,ff_z1)

CZ_TP1.add((2,3),PERM([1,0]))

CX_TP1 = pcvl.Processor("SLOS", 8)
CX_TP1.add(2, BS.H())
CX_TP1.add(0, CZ_TP1)
CX_TP1.add(2, BS.H())

pcvl.pdisplay(CZ_TP1)

In [ ]:
from perceval.algorithm import Analyzer
analyser = Analyzer(CX_TP1, [pcvl.BasicState([0,1,0,1, 0,1,1,0])],[pcvl.BasicState([0,1,1,0, 0,1,1,0])])
#pcvl.pdisplay(analyser)

In [ ]:
import time
from operator import itemgetter
CNOT = CX_TP1
depth = 2

def phaseGate(param):
    return pcvl.Circuit(2).add(1, PS(param))
theta = [pcvl.P(f"theta{i}") for i in range(depth * 3)]
x = [pcvl.P(f"x{i}") for i in range(3)]

feature_map = pcvl.Processor(pcvl.SLOSBackend(),20)
ansatz = pcvl.Processor(pcvl.SLOSBackend(),20)


for i in range(2*depth + 1):
    if i % 2 == 0:
        ansatz.add(0, BS.Ry(theta = theta[i]))
        ansatz.add(2, BS.Ry(theta = theta[i + 1])) 
    else:
        ansatz.add((0, 1, 2, 3, 2 + 2*i, 3 + 2*i, 4 + 2*i, 5 + 2*i),CNOT)

feature_map.add(0, BS.H())
feature_map.add(2, BS.H())
feature_map.add(0, phaseGate(x[0]))
feature_map.add(2, phaseGate(x[1]))
feature_map.add((0, 1, 2, 3, 12, 13, 14, 15), CNOT)
feature_map.add(2, phaseGate(x[2]))
feature_map.add((0, 1, 2, 3, 16, 17, 18, 19), CNOT)

vqc = pcvl.Processor(pcvl.SLOSBackend(), 20)
vqc.add(0, feature_map)
vqc.add(0, ansatz)
vqc_params = vqc.get_circuit_parameters()
def_input = pcvl.BasicState([1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0])

def probs_to_predictions(probs):
    return sum(itemgetter(keys)(probs))

def get_prediction(data, num_samples):
    vqc_params["x0"].set_value(2*data[0])
    vqc_params["x1"].set_value(2*data[1])
    vqc_params["x2"].set_value(2*(pi - data[0])*(pi - data[1]))

    time1 = time.time_ns() 
    vqc.with_input(def_input)
    timea = time.time_ns()
    prob_dict = vqc.probs()["results"]
    timeb = time.time_ns()
    print((timeb - timea) / 1e6)
    time1 = time.time_ns()
    prediction = probs_to_predictions(prob_dict)
    time2 = time.time_ns()
    print((time2 - time1)/1000)
    return prediction

def loss(params, data = X_train, target = y_train, num_samples = 10):
    pred_arr = np.zeros(shape = (data.shape[0]))
    for i in range(len(params)):
        vqc_params[f"theta{i}"].set_value(params[i])
    for i in range(data.shape[0]):
        pred_arr[i] = get_prediction(data[i], num_samples)
    np.clip(pred_arr,1e-8,1 - 1e-8)
    loss = -np.mean(target*np.log2(pred_arr) + (1 - target)*np.log2(1 - pred_arr))
    return loss

pcvl.pdisplay(vqc)

In [86]:
from scipy.optimize import minimize, OptimizeResult

init_params = np.random.default_rng(42).random(size = (depth*3))

klm_arr = []

def callback(intermediate_result: OptimizeResult):
    klm_arr.append(intermediate_result.fun)
    print(intermediate_result.fun)

res = minimize(loss, init_params, args = (X_train, y_train), method = "L-BFGS-B", callback = callback)

UnavailableModeException: Mode(s) 4 not available

In [ ]:
# Function to calculate accuracy
def calculate_accuracy(params, x_data, y_true):
    # Get predictions using the trained model
    predictions = np.zeros(shape = (x_data.shape[0]))
    for i in range(len(params)):
        vqc_params[f'theta{i}'].set_value(params[i])
    for i in range(x_data.shape[0]):
        predictions[i] = get_prediction(x_data[i])
    predicted_labels = (predictions >= 0.5).astype(int)
    print(predictions)
    print(predicted_labels)
    # Calculate accuracy
    accuracy = np.mean(predicted_labels == y_true)
    return accuracy
opt_params = res.x
# Calculate training accuracy
train_accuracy = calculate_accuracy(opt_params, X_train, y_train)
print(f"Training Accuracy: {train_accuracy:.4f}")

# Calculate testing accuracy
test_accuracy = calculate_accuracy(opt_params, X_test, y_test)
print(f"Testing Accuracy: {test_accuracy:.4f}")
print(y_test)
